# Photoionization

In this second module, you will calculate the K-edge photoionization cross section within the
single particle approximation and the sudden approximation for three noble gas elements: Neon,
Argon and Krypton. Further details on the theory behind this exercise can be found at: J.Phys.B:
At.Mol.Opt.Phys. 42 (2009) 023001. Many functions and routines you will use in this exercise have
already been introduced in Module 1. Therefore, it might be useful to look it up while solving the
exercise.

Import relevant libraries:

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy import integrate, special

The following functions are needed to solve the exercises

In [2]:
from sympy.physics.quantum.cg import CG

# Clebsch Gordan coefficients
def cgc(j1,m1,j2,m2,j3,m3):
    
    '''
    uses the sympy module to calculate the Clebsch Gordan coefficients
    <j1m1j2m2|j3m3>
    '''

    cg = CG(j1,m1,j2,m2,j3,m3)
    
    return cg.doit().evalf()
    

# numerical integration
def triple_trap(nr,nt,nph,intD,rf,*arg):
    
    # integration limits
    r1 = 0.
    t1 = 0.
    tf = 2*(np.pi) 
    p1 = 0.
    pf = np.pi
    
    # integration steps
    dr = (rf-r1)/nr
    dt = (tf-t1)/nt
    dp = (pf-p1)/nph  
    
    ans = 0.
    for i in range(nr-1):
        for j in range(nt-1):
            for k in range(nph-1):
                ri = r1 + dr/2 + i*dr
                tj = t1 + dt/2 + j*dt
                pk = p1 + dp/2 + k*dp
                ans += dr*dt*dp*intD(pk,tj,ri,*arg)
    
    return ans

The following python function returns the hydrogen-like wave function $\Psi_{nlm}$ for given values of $n,l,m, \theta,\phi$, $r$ and $Z_{eff}$. The function is very similar to the one you have already written in module I.


In [3]:
def Psi(n,l,m,r,phi,thet,Z):
    '''
    n: principal quantum number
    l: azimuthal quantum number
    m: magnetic quantum number
    r, phi and thet: spherical coordinates
    Z: nuclear charge
    
    return the total wave function Psi, the radial part R and the angular part Y
    '''

    # angular part -> spherical harmonics
    Y = special.sph_harm(m,l,thet,phi)

    # radial part containing the associated Laguerre polynomials
    R = np.sqrt((math.factorial(n-l-1)*(2*Z/n)**3)/(2*n*math.factorial(n+l)))
    R *= (2*r*Z/n)**l
    R *= np.exp(-Z*r/n)
    x = 2*r*Z/n
    R *= special.assoc_laguerre(x,n-l-1,2*l+1)

    psi = Y*R

    return psi, R, Y


The following functions return the value of the wavefunction for the 1s orbital of Neon ($Z_{eff}$=9.642), Argon ($Z_{eff}$=17.508) and Krypton ($Z_{eff}$=35.232). Take as a reference the functions written in module 1.

In [4]:
def Ne_1s(r,phi,theta):
    Zeff = 9.642
    return Psi(1,0,0,r,phi,theta,Zeff)[0]

def Ar_1s(r,phi,theta):
    Zeff = 17.508
    return Psi(1,0,0,r,phi,theta,Zeff)[0]

def Kr_1s(r,phi,theta):
    Zeff = 35.232
    return Psi(1,0,0,r,phi,theta,Zeff)[0]

### Photoionization: theory
The transition dipole moment of an ionization process involves a bound and a continuum state as initial and final state, respectively. If one describes the ejected electron as a free particle (hence neglecting the interaction between the ejected electron and the N − 1 electron residual ion), the final state can be expressed as a product between a bound state and a continuum state $<\Psi_{f}>=\Psi_{N-1}\phi_e$, where $\phi_e$ describes the ejected electron. The transition moment becomes $<\Psi_{N-1}\phi_e|\hat{r}|\Psi_{N}>$. In an independent electron picture, the integral simplifies to $<\phi_e|\hat{r}|\phi_i>$ where $\phi_i$ is a one-particle eigenstate of a single body Hamiltonian, i.e. the Fock Hamiltonian. It represents the initial state (i.e. the molecular orbital in the single particle picture) of the electron before being promoted into the continuum during the ionisation process. Here we also assume that the final $\Psi_{N-1}$ state is not perturbed by the loss of the electron. This approximation is usually referred to as the sudden approximation.

The free electron wave function is approximated by a plane wave, with expansion:

$$\phi_e(\vec{k},\vec{r}) = \frac{2}{\sqrt{2\pi}} \sum_{l=0}^{\infty}\sum_{m=-l}^{l} i^lR_l(kr)Y_{l}^{m}(\vec{k})Y_{l}^{m*}(\vec{r})$$ 

where $\vec{k}$ is the wave vector of the ejected electron in spherical coordinates $\vec{k}=(k,\theta_k,\phi_k)$ and $\vec{r}$ is the position vector $\vec{r}=(r,\theta,\phi)$. The radial part $R_l(kr)$ is given by the spherical Bessel function (which can be retrieved from the 'scipy.special' package by means of 'special.spherical\_jn(l,kr)'). The limit of the sum over the $l$ values is usually taken as $l_{max}$ that is large enough to reach convergence.

##### 1) Write down a function to calculate the free electron's wave function $\phi_e(k,r,\theta_k,\phi_k,\theta,\phi) =\frac{2}{\sqrt{2\pi}} \sum_{l=0}^{lmax}\sum_{m=-l}^{l} i^lR_l(kr)Y_{l}^{m}(\theta_k,\phi_k)Y_{l}^{m*}(\theta,\phi)$ for a given $k, l_{max},r, \theta$, $\phi$, $\theta_k$ and $\phi_{k}$. 

##### 2) Plot the probability density as a function of $r$ (with $r$ between 0 a.u. and 50 a.u.) of $\phi_e$ with $l_{max}=10$. Choose various values of $\theta$, $\phi$, $\theta_k$, $\phi_k$ and $k$. You might want to start with $\theta=\phi=\theta_k=\phi_k=0$ or $\theta=\pi/2$, $\phi=\pi/5$, $\theta_k=\pi/7$, $\phi_k=\pi/3$ for example.

The partial differential photoionisation cross section associated with the population of the final state $\phi_e$ is given by:

$$\frac{\partial \sigma_i^a}{\partial \theta_k \partial \phi_k}=4\pi^2\alpha\,\omega_{in}\,\delta(E_{\phi_e}-E_{\phi_i}-\omega_{in})\sum_b|d_b(k,\theta_k,\phi_k)|^2$$ 

where $\omega_{in}$ is the energy of the incoming photon, $\alpha$ is the fine structure constant (corresponding to 1/c = 1/137 in atomic units). The index b in the sum corresponds to the components of the transition dipole matrix elements. The ability to resolve $\theta_k$ and $\phi_k$ allows to perform photoelectron angular distribution spectroscopy. Here, we are only interested in the total photoionization cross section, hence we  integrate over $\theta_k$ and $\phi_k$, which leads to the integrated cross section:

$$\sigma_i^k=4\pi^2\alpha\,\omega_{in}\,\delta(E_{\phi_e}-E_{\phi_i}-\omega_{in})\sum_b|d_b(k)|^2$$

The conservation of energy enforced by the delta function imposes that $E_{\phi_e}-E_{\phi_i}=\omega_{in}$ is respected. $E_{\phi_i}$ is the binding energy of the electron whose opposite can, in the context of Koopman's theorem, be considered equal to the ionization potential IP. In the context of our model, the binding energy is equal to 

$$E_{\phi_i}=IP=-(\frac{Z_{eff}^2}{2n^2})$$ 

in atomic units ( note: $1$ au $= 27.211$ eV). On the other hand, $E_{\phi_e}$ is the kinetic energy of the free electron which, in atomic units, is equal to $E_{\phi_e}=\frac{1}{2}k^2$. All considered, we can therefore rewrite the conservation of energy relation as $\frac{1}{2}k^2+IP = \omega_{in}$. 

When integrating over $\theta_k$ and $\phi_k$, the factor $Y_{lm}(\theta_k,\phi_k)$ is integrated out, leading to $d_{b}(k)=<\phi_{e}(k)|\hat{r}_{b}|\phi_{i}>$. The value of $\sigma_i^k$ is relative to a single electron in the subshell $i$. A more practical quantity is the subshell cross section, which can be obtained by multiplying the value of $\sigma_i^k$ by the number of electrons in the shell.

In the following we will assume that our light is z-polarized. Hence $d_b=d_z$ and $\hat{r}=r_z=rcos(\theta)$


3) Write down a function that returns the real and imaginary part of the integrand needed to calculate the z component of the dipole transition moment $<\phi_e|rcos(\theta)|\phi_{1s}>$. $\phi_{1s}$ is the wave function associated to the $1s$ orbital of Neon, Argon and Krypton. Hint 1: use the function written in 1) whitout the $Y_l^m(\theta_k,\phi_k)$ factor. Hint 2: refer to the transition moment section of Module I.

4) Plot the real part of the integrand (for whichever atom you like) as a function of $r$ (in a range between 0 and 3 a.u. of length) for several values of $k$, $\theta$ and $\phi$. In particular, look at the behavior of the integrand for small and large values of $k$ (e.g. for k<1 and k>1). Use $l_{max}=10$.

5) Plot the real and imaginary part of the integrand as a function of radius (for whichever atom you like) with $k=1$ a.u. and a defined value for $\theta$ and $\phi$ (say $\theta=\pi/3$ and $\phi=\pi/4$), for various values of $l_{max}$

6) The following block calculates the Neon integrated photoionization cross section for several values of final state’ energy (i.e. with k between 0 au and 5 au) and plots σ i as a function of the photon energy. As you can see the $triple_trap ( nr, nth, nph, integrand, r f , ∗ arg )$ function is used to perform the integration according to the trapezoidal rule. We have already prepared a template of the function for you, you have only to substitute the ******* with the definitions and the names of the functions you’ve previously defined. We want the values of the energy to be expressed in eV and the value of the cross-section to be expressed in MegaBarn (Mb) (1 a.u. = 28 Mb). You can try different mesh grids for $r$, $\theta$ and $\phi$. 


In [ ]:
# choose lmax appropriately using the result in 5) 
lmax = 4

# Ionization energy in eV
IE_Ne_1s = *******     #substitute with the binding energy of Neon (eV)

# vector for sigma and w_in (or Etot)
Ne1s_sig1 = []    #list of the cross section value for each transition total energy -- creation
Ne_etot_1s = []   #list of transition total energy -- creation

# value for k in au
k_vector = np.linspace(0,5,5)

# mesh grids
rf = 10
nr = 100
nth = 10
nph = 10

# calculate total cross section for several values of k 
# energies and the fine structure constant alpha must be given in eV
for k in k_vector:
  ek = ******  #substitute with the expression to calculate the energy of the final state
  Ne_etot_1s.append(*****)  #substitute with the expression to calculate the total energy.
  # Integration using trapezoide rule  
  real = triple_trap(nr,nth,nph,******,rf,k,True) #substitute the name of the Ne function you've written in block 3
  imag = triple_trap(nr,nth,nph,******,rf,k,False) #substitute the name of the Ne function you've written in block 3
  d2 = abs(complex(real,imag))**2   #module squared of the dipole transition moment
  Ne1s_sig1.append(**********) #substitute the definition of cross section. Remember to account for all the electrons of the shell and the units.

#plot


plt.plot(Ne_etot_1s,Ne1s_sig1,'r')
plt.show()


You might have noticed that despite the low value of $l{max}$ needed to calculate $\phi_e$, the numerical integration takes quite some time. The computational time will drastically increase if we want to calculate the cross section for larger electron energy (hence larger photone energies $\omega_{in}$ ). Clearly our “brute” force approach to the calculation of the dipole transition moment is not efficient. 

The problem can be simplified by taking advantage of the properties of the spherical harmonics. The transition dipole moment can be expressed as:

$$<\phi_e(r)\phi_e(\theta,\phi)|rcos(\theta)|\phi_i(r)\phi_i(\theta,\phi)>$$

when separating the radial part, the expression becomes:

$$\sum_{l=0}^{lmax}\sum_{ml=-l}^{+l}<i^lR_l(kr)|r|R_{\phi_i}(r)><Y_{l}^{m}(\theta,\phi)|cos(\theta)|Y_{\phi_i}(\theta,\phi)>$$

expressing $cos(\theta)$, the angular part of $r_z$, in spherical harmonic basis yields $cos(\theta)=\sqrt{\frac{4\pi}{3}}Y_{10}(\theta,\phi)$. The integral becomes:

$$\sum_{l=0}^{lmax}\sum_{ml=-l}^{+l}<i^lR_l(kr)|r|R_{\phi_i}(r)><Y_{l}^{m}|\sqrt{\frac{4\pi}{3}}Y_{1}^{0}|Y_{\phi_i}>$$

the factor involving the 3 spherical harmonics can be simplified using the $\textit{integral over three spherical harmonics}$ relation from angular momentum theory which states:

$$ \sqrt{\frac{(2l+1)(2l_{\omega}+1)}{4\pi(2l_{\phi_i}+1)}}<l,0,l_{\omega},0|l_{\phi_i},0><l,m,l_{\omega},m_{\omega}|l_{\phi},m_{\phi}> $$

the factors $<l, ml|l_{\omega}, ml_{\omega}|l_{\phi_i}, ml_{\phi_i}>$ are the Clebsh-Gordan coefficients ($cgc$), where $l, ml$ are the angular momentum quantum numbers of the ejected electron, $l_{\omega}, ml_{\omega}$ are the angular momentum quantum numbers of the photon, and $l_{\phi_i}, ml_{\phi_i}$ are the angular momentum quantum numbers of the initial orbital $\phi_i$. 

The function $cgc$ gives the result for a set of angular momentum.
For example:

In [ ]:
print(cgc(2,0,1,0,1,0))

-0.632455532033676


For a $z$ polarized light: $l_{\omega}=1$ and $ml_{\omega}=0$. If $\phi_i$ is a $2p_{0}$ orbital: $l_{2p_0}=1$, $ml_{2p_0}=0$

7) Try to calculate the $cgc$ coefficient for different value of $l$ and $ml$. You should recover the selection rules $\Delta l = (l_{\phi_i}-l)= \pm 1$ and $\Delta ml = (ml_{\phi_i} - ml)= 0$ for a one photon transition, which arise from symmetry consideration.

This reformulation will considerably reduce the complexity of the integral. Indeed, we only need to perform the integration over the radial part while the angular part is given by the $cgc$ coefficients.

8) Write down a function to calculate the integrand of the radial part $i^l<R_l(kr)|r|R_{1s}(r)>$ of Ne, Ar and Kr.
Note: the radial part of $\phi_e$ and the radial part of $1s$ are different functions altought the same letter $R$ is used.

9) Calculate the integrated cross section of Ne for various photon energies. Produce a plot similar to the one shown in exercise 6). The upper value for the radial integration ($r_f=20.0$) and the k-vector range are already given. However, feel free to play around with them. 
Hint: when structuring your loops consider that the terms in the plane wave expansion will only be non-zero for certain value of $ml$ and $l$ (look at the $cgc$ in exercise 7).



10) Describe the characteristic shape of the photoionization cross section.

11) Plot the cross sections of Neon, Argon and Krypton in the same figure. What do you observe? Compare the peak values and the position of the maxima of the cross sections for various elements. What are the experimental parameters that you would need to change to selectively ionise Ne, Ar, Kr in an experiment of a mixed gas?

12) Our model predicts $1s$ binding energies of 1264.4 eV, 4168,8 eV and 16881.6 eV for Neon, Argon and Krypton respectively. The exact values energies are 870,2 eV (Ne), 3205.9 eV (Ar) and 14326 (Kr). This indicates that our model is not reproducing very precisely the binding energies. Why is it so? What would you do to improve the model? What effects are we missing here?

The cross section can also be expressed as the ratio of the transition rate between the initial and final states $\Gamma_{FI}$ and the photon flux 

$$J_{EM}: \sigma = \frac{\Gamma_{FI}}{J_{EM}}$$

The photon flux is given by the number of photons per unit time and unit area. Multiplying the rate by the duration of the pulse, the probability of transition during a given pulse is retrieved. 

To put the cross section into perspective, let’s estimate the K-shell X-Ray absorption probability of Ne when exposed to a X-Ray pulse from a third-generation synchrotron radiation source and a X-Ray Free Electron Laser. The maximum value (0.000238Mb) of the calcuated Ne cross section is found at an energy of 1684.2 eV. Let’s consider a monocromatized x-ray pulse at a synchrotron (bandwidth 1eV) containing about $10^6$ photons. The pulses of a synchrotron are typicall 1 ns long. 

13) If such a pulse is focused to a spot of $10\mu m \times 10 \mu m$, what is the absorption probability per pulse and per atom in the focus? At an X-ray free-electron laser (XFEL) we can have around $10^{11}$ photons per pulse with a focus of 1 μm x 1 μm. Suppose a pulse duration of 10 fs (1 fs $= 10^{−15}$ s). What is the transition probability then? Comment on the difference of both results.

